In [1]:
import src.kgbench as kg
from src.rgcn_explainer_utils import *

data = kg.load('mutag', torch=True, final=True)
data = prunee(data, 2)
get_relations(data)
relations = [data.i2rel[i][0] for i in range(len(data.i2rel))]

/usr/local/Caskroom/miniconda/base/envs/RGCNExplainer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/RGCN_stuff/src/kgbench.py:247: UserWarning: The validation data is not added to the training data. For AIFB, MUTAG, BGS and AM, the correct evaluation is to combine train and validation for the final evaluation run.Set include_val to True when loading the data.
  warnings.warn('The validation data is not added to the training data. For AIFB, MUTAG, BGS and AM, '


loaded data mutag (0.3235s).


In [2]:
val = data.withheld[:,0]
most_common_node = {}
for i in val:
    sub_edges, neighborhoods, sub_edges_tensor = find_n_hop_neighbors(data, 2,int(i), adj=True)

    counter = Counter([num for tup in sub_edges for num in tup]).most_common(1)[0][0]
    most_common_node[int(i)] = counter

In [3]:
most_common_node

{10670: 13116,
 22269: 23076,
 16607: 23076,
 21167: 23076,
 16144: 23076,
 7330: 23076,
 14186: 23076,
 18013: 23076,
 4737: 23076,
 13391: 23076,
 1019: 13116,
 7212: 13116,
 9982: 23076,
 20452: 13116,
 18610: 23076,
 1950: 23076,
 1379: 23076,
 5111: 23076,
 14178: 23076,
 17744: 23076,
 16130: 23076,
 10348: 13116,
 9191: 23076,
 19068: 23076,
 10022: 23076,
 9688: 13116,
 8338: 23076,
 15502: 23076,
 1148: 23076,
 1943: 23076,
 9715: 23076,
 6465: 23076,
 4474: 23076,
 9734: 23076,
 17429: 23076,
 2950: 23076,
 1700: 23076,
 8171: 23076,
 8873: 23076,
 20780: 23076,
 1063: 23076,
 10291: 23076,
 8701: 13116,
 16507: 23076,
 14907: 23076,
 14914: 23076,
 21759: 23076,
 11614: 13116,
 19062: 13116,
 20543: 23076,
 1917: 23076,
 19854: 23076,
 19886: 23076,
 4000: 23076,
 6581: 23076,
 11771: 23076,
 18261: 23076,
 4246: 23076,
 9460: 23076,
 19975: 23076,
 12812: 23076,
 14411: 23076,
 13126: 23076,
 14676: 23076,
 5054: 23076,
 3471: 23076,
 16282: 23076,
 10172: 23076}

In [4]:
def frequency_relations(data):
    freq = Counter(data.triples[:,1].tolist())
    sorted_freq = {data.i2r[k]: v for k, v in sorted(freq.items(), key=lambda item: item[1], reverse=True)}
    return sorted_freq
f = frequency_relations(data)
print(f)

{'http://www.w3.org/1999/02/22-rdf-syntax-ns#type': 7942, 'http://dl-learner.org/carcinogenesis#inBond': 6370, 'http://dl-learner.org/carcinogenesis#hasBond': 3185, 'http://dl-learner.org/carcinogenesis#hasAtom': 3133, 'http://dl-learner.org/carcinogenesis#charge': 3133, 'http://dl-learner.org/carcinogenesis#hasStructure': 1197, 'http://dl-learner.org/carcinogenesis#amesTestPositive': 340, 'http://dl-learner.org/carcinogenesis#salmonella': 305, 'http://dl-learner.org/carcinogenesis#cytogen_ca': 293, 'http://dl-learner.org/carcinogenesis#cytogen_sce': 283, 'http://dl-learner.org/carcinogenesis#mouse_lymph': 203, 'http://dl-learner.org/carcinogenesis#salmonella_n': 61, 'http://dl-learner.org/carcinogenesis#chromaberr': 55, 'http://dl-learner.org/carcinogenesis#chromex': 35, 'http://dl-learner.org/carcinogenesis#drosophila_slrl': 25, 'http://dl-learner.org/carcinogenesis#drosophila_rt': 25, 'http://www.w3.org/2000/01/rdf-schema#subClassOf': 19, 'http://www.w3.org/2000/01/rdf-schema#domain

In [5]:
def degree_distribution(data):
    edge_index = edge_index_oneadj(data.triples)
    degree = []
    for i in range(data.num_entities):
        _,n,_ = find_n_hop_neighbors(edge_index, 0, i)
        degree.append(len(n))
    degree_mean = np.mean(degree)
    return degree_mean

degree_distribution(data)

KeyboardInterrupt: 

In [ ]:
def object_type(v,h,data, relation_id = None,type=True):
    ''' Get the object class of a specific relation'''
    if type:
        relation_id = [i for i in range(data.num_relations) if 'type' in data.i2r[i]][-1]
    output_indices_v, output_values, value_indices = select_relation(v, data.num_entities, relation_id)
    output_indices_h, output_values, value_indices = select_relation(h, data.num_entities, relation_id)
    objects_types = match_to_triples(output_indices_v, output_indices_h,data, sparse=False)
    list = []
    for i in objects_types:
        list.append(data.i2e[i[2]][0])#.split('#')[1])
    result = Counter(list)
    return result

for node_idx in data.withheld[:,0]:
    h, v = torch.load(f'chk/aifb_chk/hops_2_size_5e-05_lr_0.1_ent_-1_type_1_threshold_0.5_init_const_exp_/masked_adj/masked_ver{node_idx}'), torch.load(f'chk/aifb_chk/hops_2_size_5e-05_lr_0.1_ent_-1_type_1_threshold_0.5_init_const_exp_/masked_adj/masked_hor{node_idx}')
    print(f'node {node_idx}:', object_type(v,h,data, 39))